In [1]:
import pickle
import numpy as np
from scipy.interpolate import NearestNDInterpolator, LinearNDInterpolator

https://stackoverflow.com/questions/78644870/re-sampling-2d-dem-data-into-a-regular-local-level-array

In [2]:
# Fake DEM data (don't know how to warp it in the way the TED output is, so just adding some rand) This
# could use a rectilinear interpolator, but I believe my data cannot (warped in E as a function of N and
# vice versa).
e_dim = 7000
n_dim = 6000
e_rng = np.arange(-e_dim/2, e_dim/2, dtype='float32')*72 + np.random.randint(-100, high=101, size=e_dim)
n_rng = np.arange(-n_dim/2, n_dim/2, dtype='float32')*95 + np.random.randint(-100, high=101, size=n_dim)
dem_u = np.random.randint(-100, high=4500, size=(n_dim,e_dim))
row, col = np.meshgrid(n_rng, e_rng, indexing='ij')
ENU = np.dstack((row,col,dem_u))


In [8]:
# looking for a faster approach to lookup Nx2 arrays of E/N points that are on a regular sample grid for # rastered final output

%timeit -n 1 -r 30 interp = NearestNDInterpolator((ENU[:,:,0].flatten(), ENU[:,:,1].flatten()), ENU[:,:,2].flatten())
# times about 12-13s on my hardware--pretty darn slow.
# 11.8 s ± 488 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)

11.8 s ± 488 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [17]:
%timeit -n 1 -r 30 interp = NearestNDInterpolator((ENU[:,:,0].ravel(), ENU[:,:,1].ravel()), ENU[:,:,2].ravel())
# 11.7 s ± 630 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)

11.7 s ± 630 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [11]:
11.7 - 3.67

8.03

In [4]:
%timeit -n 1 -r 30 interp = NearestNDInterpolator((ENU[:,:,0].ravel(), ENU[:,:,1].ravel()), ENU[:,:,2].ravel(), tree_options={"compact_nodes": False, "balanced_tree": False, "leafsize": 64})
# 3.67 s ± 254 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)

3.67 s ± 254 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [12]:
(11.7 + 4.36) / (3.67 + 5.47)

1.757111597374179

In [14]:
(11.7 + 4.36) - (3.67 + 5.47)

6.919999999999998

In [13]:
1/1.757111597374179

0.5691158156911582

In [6]:
interp = NearestNDInterpolator((ENU[:,:,0].ravel(), ENU[:,:,1].ravel()), ENU[:,:,2].ravel(), tree_options={"compact_nodes": False, "balanced_tree": False, "leafsize": 64})

In [9]:
interp = NearestNDInterpolator((ENU[:,:,0].flatten(), ENU[:,:,1].flatten()), ENU[:,:,2].flatten())

In [10]:
interval_m = 11338.7 # Just a coarse starting sample dimension
samp_offsets_m = np.array([-150*1852+i*interval_m for i in range(2500) ] )
tiles_x, tiles_y = np.meshgrid(samp_offsets_m, samp_offsets_m, indexing='xy' )
pTL = np.asarray( [[*tiles_x.flatten()],    #E pos
                   [*tiles_y.flatten()]])   #N pos

%timeit -n 1 -r 30 interp(pTL[0], pTL[1])
# Default: 4.36 s ± 149 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)
# Tweak: 5.47 s ± 139 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)

4.36 s ± 149 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)
